<a href="https://colab.research.google.com/github/tgwon/Recruitment_Text_Mining/blob/main/Code/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT를 활용하여 한국어 개체명 인식기 만들기

## 개체명 인식 데이터 불러오기 및 전처리

개체명 인식을 위한 데이터를 다운 받습니다.

In [ ]:
!wget https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data

--2023-11-16 04:04:04--  https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data [following]
--2023-11-16 04:04:04--  https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16945023 (16M) [text/plain]
Saving to: ‘train_data’

train_data          100%[===================>]  16.16M  --.-KB/s    in 0.1s    

2023-11-16 04:04:05 (129 MB/s) - ‘train_data’ saved [16945023/16945

분석에 필요한 모듈들을 임포트 합니다.

In [ ]:
!pip install transformers
!pip install sentencepiece
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import BertTokenizer
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
from keras.utils import pad_sequences
#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
from transformers import TFBertModel

train 데이터를 불러 오겠습니다.

In [ ]:
train = pd.read_csv("train_data", names=['src', 'tar'], sep="\t")
train = train.reset_index()
train

,index,src,tar
0,1,비토리오,PER_B
1,2,양일,DAT_B
2,3,만에,-
3,4,영사관,ORG_B
4,5,감호,CVL_B
...,...,...,...
769060,2,어째,-
769061,3,뭔가,-
769062,4,수상쩍은,-
769063,5,좌담,-


train 데이터에 마침표가 이상한 것들이 많아서 확실하게 .으로 수정해 주겠습니다.

In [ ]:
train['src'] = train['src'].str.replace("．", ".", regex=False)

In [ ]:
train.loc[train['src']=='.']

,index,src,tar
15,6,.,-
30,15,.,-
40,10,.,-
77,24,.,-
96,19,.,-
...,...,...,...
769013,11,.,-
769036,23,.,-
769053,17,.,-
769058,5,.,-


데이터를 전처리 해주겠습니다.  
한글, 영어, 소문자, 대문자, . 이외의 단어들을 모두 제거하겠습니다.

In [ ]:
train['src'] = train['src'].astype(str)
train['tar'] = train['tar'].astype(str)

train['src'] = train['src'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', "", regex=True)

데이터를 리스트 형식으로 변환합니다.

In [ ]:
data = [list(x) for x in train[['index', 'src', 'tar']].to_numpy()]

데이터를 잘 보면 (인덱스, 단어, 개체) 로 이루어 진 것을 알 수 있습니다.  
인덱스가 1,2,3,4,5.. 이렇게 이어지다가 다시 1,2,3,4, 로 바뀌는데 숫자가 바뀌기 전까지가 한 문장을 의미합니다.

In [ ]:
print(data[:20])

[[1, '비토리오', 'PER_B'], [2, '양일', 'DAT_B'], [3, '만에', '-'], [4, '영사관', 'ORG_B'], [5, '감호', 'CVL_B'], [6, '용퇴', '-'], [7, '항룡', '-'], [8, '압력설', '-'], [9, '의심만', '-'], [10, '가율', '-'], [1, '이', '-'], [2, '음경동맥의', '-'], [3, '직경이', '-'], [4, '8', 'NUM_B'], [5, '19mm입니다', 'NUM_B'], [6, '.', '-'], [1, '9세이브로', 'NUM_B'], [2, '구완', '-'], [3, '30위인', 'NUM_B'], [4, 'LG', 'ORG_B']]


라벨들을 추출하고, 딕셔너리 형식으로 저장하도록 하겠습니다.

In [ ]:
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

In [ ]:
print(label_dict)

{'PER_B': 0, 'DAT_B': 1, '-': 2, 'ORG_B': 3, 'CVL_B': 4, 'NUM_B': 5, 'LOC_B': 6, 'EVT_B': 7, 'TRM_B': 8, 'TRM_I': 9, 'EVT_I': 10, 'PER_I': 11, 'CVL_I': 12, 'NUM_I': 13, 'TIM_B': 14, 'TIM_I': 15, 'ORG_I': 16, 'DAT_I': 17, 'ANM_B': 18, 'MAT_B': 19, 'MAT_I': 20, 'AFW_B': 21, 'FLD_B': 22, 'LOC_I': 23, 'AFW_I': 24, 'PLT_B': 25, 'FLD_I': 26, 'ANM_I': 27, 'PLT_I': 28, '[PAD]': 29}


In [ ]:
print(index_to_ner)

{0: 'PER_B', 1: 'DAT_B', 2: '-', 3: 'ORG_B', 4: 'CVL_B', 5: 'NUM_B', 6: 'LOC_B', 7: 'EVT_B', 8: 'TRM_B', 9: 'TRM_I', 10: 'EVT_I', 11: 'PER_I', 12: 'CVL_I', 13: 'NUM_I', 14: 'TIM_B', 15: 'TIM_I', 16: 'ORG_I', 17: 'DAT_I', 18: 'ANM_B', 19: 'MAT_B', 20: 'MAT_I', 21: 'AFW_B', 22: 'FLD_B', 23: 'LOC_I', 24: 'AFW_I', 25: 'PLT_B', 26: 'FLD_I', 27: 'ANM_I', 28: 'PLT_I', 29: '[PAD]'}


데이터를 문장들과 개체들로 분리합니다.  
tups[0], tups[1],... 에 각각의 문장에 해당하는 단어와 개체 번호가 저장이 되게 됩니다.

In [ ]:
tups = []
temp_tup = []
temp_tup.append(data[0][1:])
sentences = []
targets = []
for i, j, k in data:

  if i != 1:
    temp_tup.append([j,label_dict[k]])
  if i == 1:
    if len(temp_tup) != 0:
      tups.append(temp_tup)
      temp_tup = []
      temp_tup.append([j,label_dict[k]])

tups.pop(0)

[['비토리오', 'PER_B']]

In [ ]:
print(tups[0], tups[1])

[['비토리오', 0], ['양일', 1], ['만에', 2], ['영사관', 3], ['감호', 4], ['용퇴', 2], ['항룡', 2], ['압력설', 2], ['의심만', 2], ['가율', 2]] [['이', 2], ['음경동맥의', 2], ['직경이', 2], ['8', 5], ['19mm입니다', 5], ['.', 2]]


tups를 보면 [(단어, 개체), (단어, 개체), (단어, 개체)]의 형식으로 저장이 되어 있는데, 이거를 (단어, 단어, 단어, 단어), (개체, 개체, 개체, 개체) 형식으로 변환하도록 하겠습니다.


In [ ]:
sentences = []
targets = []
for tup in tups:
  sentence = []
  target = []
  sentence.append("[CLS]")
  target.append(label_dict['-'])
  for i, j in tup:
    sentence.append(i)
    target.append(j)
  sentence.append("[SEP]")
  target.append(label_dict['-'])
  sentences.append(sentence)
  targets.append(target)

In [ ]:
sentences[0]

['[CLS]',
 '비토리오',
 '양일',
 '만에',
 '영사관',
 '감호',
 '용퇴',
 '항룡',
 '압력설',
 '의심만',
 '가율',
 '[SEP]']

In [ ]:
targets[0]

[2, 0, 1, 2, 3, 4, 2, 2, 2, 2, 2, 2]

## 버트 인풋 만들기

구글의 multilinguial-bert를 활용하도록 하겠습니다.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

(…)ultilingual-cased/resolve/main/vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

(…)ingual-cased/resolve/main/tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

(…)tilingual-cased/resolve/main/config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
tokenizer.tokenize("대한민국 만세.")

['대한민국', '만', '##세', '.']

여기서부터가 중요한데, 문장을 토크나이징 하고 개체(target)을 토크나이징 한 문장에 맞추도록 하겠습니다.  
문장 "대한민국 만세." 는 사실 (대한민국, 개체1), (만세., 개체2) 을 가지고 있는데 토크나이징을 하면 '▁대한민국', '▁만', '세', '.' 로 토크나이징이 됩니다.  
여기서 그렇다면 ( ▁대한민국, 개체1) , (▁만, 개체2), (세, 개체2), (., 개체 2) 와 같은 방식으로 각 개체를 부여해주어야 합니다.

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []

  for word, label in zip(sentence, text_labels):

    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    tokenized_sentence.extend(tokenized_word)
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
                              tokenize_and_preserve_labels(sent, labs)
                              for sent, labs in zip(sentences, targets)]

In [ ]:
print(tokenized_texts_and_labels[:2])
# [(문장, 개체들), (문장, 개체들),...] 형식으로 저장되어 있음.

[(['[CLS]', '비', '##토', '##리', '##오', '양', '##일', '만', '##에', '영', '##사', '##관', '감', '##호', '용', '##퇴', '항', '##룡', '압', '##력', '##설', '의', '##심', '##만', '가', '##율', '[SEP]'], [2, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), (['[CLS]', '이', '음', '##경', '##동', '##맥', '##의', '직', '##경', '##이', '8', '19', '##mm', '##입', '##니다', '.', '[SEP]'], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2])]


(문장, 개체들), (문장, 개체들) 을 [문장, 문장, 문장, ...] , [개체들, 개체들 개체들,,,,]로 분리해주도록 하겠습니다.

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
tokenized_texts[1]

['[CLS]',
 '이',
 '음',
 '##경',
 '##동',
 '##맥',
 '##의',
 '직',
 '##경',
 '##이',
 '8',
 '19',
 '##mm',
 '##입',
 '##니다',
 '.',
 '[SEP]']

In [ ]:
labels[1]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2]

문장의 길이가 상위 2.5%(88) 인 지점을 기준으로 문장의 길이를 정하도록 하겠습니다.  
만약 문장의 길이가 88보다 크면 문장이 잘리게 되고, 길이가 88보다 작다면 패딩이 되어 모든 문장의 길이가 88로 정해지게 됩니다.

In [ ]:
print(np.quantile(np.array([len(x) for x in tokenized_texts]), 0.975))
max_len = 88
bs = 32

96.0


버트에 인풋으로 들어갈 train 데이터를 만들도록 하겠습니다.  
버트 인풋으로는   
input_ids : 문장이 토크나이즈 된 것이 숫자로 바뀐 것,   
attention_masks : 문장이 토크나이즈 된 것 중에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 마스킹  
[input_ids, attention_masks]가 인풋으로 들어갑니다.

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype = "int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")

In [ ]:
input_ids[1]

array([   101,   9638,   9634,  31720,  18778, 118915,  10459,   9707,
        31720,  10739,    129,  10270,  17525,  58303,  48345,    119,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

정답에 해당하는 개체들을 만들어 보겠습니다.  
패딩에 해당하는 부분은 label_dict([PAD])(29)가 들어가게 되겠습니다.

In [ ]:
tags = pad_sequences([lab for lab in labels], maxlen=max_len, value=label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [ ]:
tags[1]

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  5,  5,  5,  5,  5,  2,  2,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29])

어텐션 마스크를 만들어 주겠습니다.

In [ ]:
attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in input_ids])

train 데이터에서 10% 만큼을 validation 데이터로 분리해 주겠습니다.

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)

In [ ]:
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

## 개체명 인식 모델 만들기

In [ ]:
# TPU 작동을 위해 실행
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)

  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)

  nr_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

## 훈련 및 성능 검증

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  nr_model = create_model()
  nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 88)]         0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 88)]         0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 88,                                          

1843/1843 [==============================] - 314s 120ms/step - loss: 0.2239 - sparse_categorical_accuracy: 0.9357 - val_loss: 0.1558 - val_sparse_categorical_accuracy: 0.9527
Epoch 2/3
1843/1843 [==============================] - 168s 91ms/step - loss: 0.1415 - sparse_categorical_accuracy: 0.9568 - val_loss: 0.1424 - val_sparse_categorical_accuracy: 0.9570
Epoch 3/3
1843/1843 [==============================] - 168s 91ms/step - loss: 0.1146 - sparse_categorical_accuracy: 0.9644 - val_loss: 0.1390 - val_sparse_categorical_accuracy: 0.9587


In [ ]:
# 만약 TPU를 사용하지 않고 GPU를 사용한다면
#nr_model = create_model()
#nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
y_predicted = nr_model.predict([val_inputs, val_masks])

205/205 [==============================] - 19s 64ms/step


In [ ]:
f_label = [i for i, j in label_dict.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(val_tags).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist()]
f_label.remove("[PAD]")

각 개체별 f1 score를 측정하도록 하겠습니다.  
참고로 micro avg는 전체 정답을 기준으로 f1 score을 측정한 것이며,  
macro avg는 각 개체별 f1 score를 가중평균 한 것입니다.

In [ ]:
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

              precision    recall  f1-score   support

       PER_B       0.84      0.85      0.84     11508
       DAT_B       0.89      0.84      0.87      4354
           -       0.95      0.95      0.95    136806
       ORG_B       0.84      0.82      0.83     13511
       CVL_B       0.75      0.81      0.78     15837
       NUM_B       0.94      0.91      0.93     11161
       LOC_B       0.81      0.79      0.80      5912
       EVT_B       0.80      0.79      0.80      3887
       TRM_B       0.86      0.67      0.75      6052
       TRM_I       0.39      0.46      0.42       670
       EVT_I       0.78      0.78      0.78      1833
       PER_I       0.71      0.72      0.71      1578
       CVL_I       0.52      0.37      0.43       869
       NUM_I       0.71      0.73      0.72      1538
       TIM_B       0.81      0.89      0.85       630
       TIM_I       0.91      0.84      0.87       219
       ORG_I       0.53      0.75      0.62      1481
       DAT_I       0.80    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 실제 데이터로 실습하기

In [ ]:
def ner_inference(test_sentence):


  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(index_to_ner[label_idx])

  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))

In [ ]:
ner_inference("Database, Data Warehouse에 대한 기본적인 이해가 있으신 분. ML 학습을 위한 데이터셋 구축 경험이 있으신 분. 모바일 게임에 대한 이해도가 높으신 분. 확장 가능한 데이터 아키텍처의 구성에 대한 관심이 있으신 분. 데이터 마트를 설계 해보신 경험 또는 그에 준하는 지식이 있으신 분")

1/1 [==============================] - 5s 5s/step
-	Database
-	,
TRM_I	Data
TRM_I	Ware
TRM_I	house
TRM_I	에
-	대한
-	기
-	본
-	적인
-	이
-	해
-	가
-	있
-	으
-	신
-	분
-	.
TRM_B	ML
-	학
-	습
-	을
-	위한
TRM_I	데
TRM_I	이터
TRM_I	셋
-	구
-	축
-	경
-	험
-	이
-	있
-	으
-	신
-	분
-	.
TRM_B	모
TRM_B	바
TRM_B	일
TRM_I	게임
TRM_I	에
-	대한
-	이
-	해
-	도가
-	높
-	으
-	신
-	분
-	.
-	확
-	장
-	가
-	능한
-	데
-	이터
TRM_I	아
TRM_I	키
TRM_I	텍
TRM_I	처
TRM_I	의
-	구
-	성
-	에
-	대한
-	관
-	심
-	이
-	있
-	으
-	신
-	분
-	.
-	데
-	이터
-	마
TRM_I	트를
-	설
-	계
-	해
-	보
-	신
-	경
-	험
-	또는
-	그


In [ ]:
ner_inference('SQL 쿼리 작성이 가능하시분')

1/1 [==============================] - 1s 1s/step
-	SQL
-	쿼
-	리
-	작
-	성이
-	가
-	능
-	하
-	시
-	분


In [ ]:
ner_inference('파이썬을 활용한 데이터 시각화 툴을 사용하실 수 있는 분')

1/1 [==============================] - 1s 1s/step
TRM_B	파
TRM_B	이
TRM_B	썬
TRM_B	을
-	활
-	용한
-	데
-	이터
-	시
-	각
-	화
TRM_I	툴
TRM_I	을
-	사
-	용
-	하
-	실
-	수
-	있는
-	분


In [ ]:
ner_inference(' Sequence labeling, text segmentation 연구 경험이 있는 분, Multi modality(Image, Text 등) 기반 머신러닝 알고리즘 개발 경험이 있는 분, 머신러닝 관련 대회 참가 경험이 있는 분, 머신러닝 관련 국제학술대회에 논문을 게재한 경험이 있는 분, 관련 분야 석사학위 이상을 소지하신 분')

1/1 [==============================] - 1s 754ms/step
-	Se
-	quence
TRM_I	label
TRM_I	ing
TRM_I	,
TRM_I	text
TRM_I	segment
TRM_I	ation
-	연구
-	경
-	험
-	이
-	있는
-	분
-	,
-	Multi
TRM_I	moda
TRM_I	lity
TRM_I	(
TRM_I	Image
TRM_I	,
TRM_I	Text
-	등
-	)
-	기
-	반
TRM_B	머신
TRM_B	러
TRM_B	닝
TRM_I	알
TRM_I	고
TRM_I	리
TRM_I	즘
-	개발
-	경
-	험
-	이
-	있는
-	분
-	,
TRM_B	머신
TRM_B	러
TRM_B	닝
-	관련
-	대회
-	참
-	가
-	경
-	험
-	이
-	있는
-	분
-	,
TRM_B	머신
TRM_B	러
TRM_B	닝
-	관련
-	국제
-	학
-	술
-	대
-	회
-	에
-	논
-	문을
-	게
-	재
-	한
-	경
-	험
-	이
-	있는
-	분
-	,
-	관련
-	분
-	야
-	석
-	사
-	학
-	위
-	이상
-	을
-	소
-	지
-	하


In [ ]:
ner_inference('AWS, Azure, GCP, Snowflake, Databricks 등 클라우드 기반의 데이터 플랫폼 경험, Tableau, Power BI, Grafana, Streamlit 등을 활용한 데이터 분석/시각화 툴 개발 경험 ,버전 관리와 협업을 위한 Git/Github 경험, 소프트웨어 컨테이너화 및 오케스트레이션을 위한 Docker/Docker Compose 경험, 의료/헬스케어 데이터 관련 경험,Data/AI 관련 분야 논문 출판 기록 또는 학술적 성과')

1/1 [==============================] - 1s 1s/step
TRM_B	A
TRM_B	WS
TRM_B	,
TRM_B	Azur
TRM_B	e
TRM_B	,
TRM_B	GC
TRM_B	P
TRM_B	,
TRM_B	Snow
TRM_B	f
TRM_B	lake
TRM_B	,
TRM_B	Data
TRM_B	bric
TRM_B	ks
-	등
-	클
-	라
TRM_B	우
-	드
-	기
-	반
-	의
-	데
-	이터
-	플
-	랫
-	폼
-	경
-	험
-	,
TRM_B	Table
TRM_B	au
TRM_B	,
TRM_B	Power
TRM_I	B
TRM_I	I
TRM_I	,
TRM_B	Graf
TRM_B	ana
TRM_B	,
TRM_B	Stream
TRM_B	lit
-	등을
-	활
-	용한
-	데
-	이터
TRM_I	분
TRM_I	석
-	/
-	시
-	각
-	화
TRM_I	툴
-	개발
-	경
-	험
-	,
-	버
-	전
-	관
-	리
-	와
-	협
-	업을
-	위한
TRM_B	G
TRM_B	it
TRM_B	/
TRM_B	G
TRM_B	ith
TRM_B	ub
-	경
-	험
-	,
TRM_B	소
TRM_B	프트
TRM_B	웨
-	어
-	컨
-	테
-	이
-	너
-	화
